In [12]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import pandas as pd
from sklearn import svm, model_selection, metrics, linear_model, tree
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.svm import SVC, SVR, LinearSVR
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectPercentile, SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, LogisticRegression

from lightgbm import LGBMClassifier, LGBMRegressor
import sklearn.feature_selection as feature_selection
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Importing libraries for building the neural network
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from keras import regularizers      #for l2 regularization
from keras.wrappers.scikit_learn import KerasRegressor 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.utils import class_weight

In [13]:
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.svm import SVC, SVR, LinearSVR
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectPercentile, SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, LGBMRegressor
import sklearn.feature_selection as feature_selection
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

In [101]:
acids = ["R", "H", "K", "D", "E", "S", "T", "N", "Q", "C", "U", "G","P", "A", "I", "L", "M", "F","W", "Y", "V"]
def seq_to_vec(seq):
    """amino acid sequence to a number (index in the array of acids)
       H - 1 index -> 1
       R - 0 index -> 0
    """
    vec = []
    for word in seq:
        vec.append(acids.index(word)) 
    return np.array(vec)

def hot_encoder(elem):
    """amino acid sequence to a hot vector of dim 21 (one in the place of index-number in the array of acids)
       H - 1 index -> 010000000000000000000
    """
    hot_vector = np.zeros(21)
    index = acids.index(elem)
    hot_vector[index] = 1
    return hot_vector

def hot_encoder_group(elem):
    """amino acid sequence to a hot vector of dim 21 (number of group in the place of index-number in the array of acids)
       D - 3 index and belongs to 2-d group; i -> 000200000000000000000;
    """
    hot_vector = np.zeros(21)
    index = acids.index(elem)
    group_index = group_amino(index + 1)
    hot_vector[index] = group_index + 1
    return hot_vector

def seq_to_hot(seq):
    vec = []
    for word in seq:
        vec.append(hot_encoder(word)) 
    return np.array(vec)

def seq_to_hot_tensor(seq):
    vec = []
    for word in seq:
        #!!!
        vec.append(hot_encoder(word)) 
    return np.array(vec).reshape(4, 21)

def hot_group_amino(letter):
    hot_vector = np.zeros(4)
    index = acids.index(letter)
    group_index = group_amino(index + 1) - 1
    hot_vector[group_index] = 1
    return hot_vector

def seq_to_hot_group(seq):
    vec = []
    for word in seq:
        vec.append(hot_encoder_group(word)) 
    return np.array(vec)

def global_hot(seq):
    global_hot = []
    for elem in seq:
#         print("elem",elem)
        global_hot += hot_encoder(elem).tolist()
    for elem in seq:
        global_hot += hot_group_amino(elem).tolist()
    return np.array(global_hot)
#     result = np.stack(global_hot, axis = 0)
#     return result

def group_amino(idx):
    """return chemical groupping for as side chain groups
       check this: https://commons.wikimedia.org/wiki/File:Amino_Acids-wide.svg
    """
    if (idx == 0 or idx == 1 or idx == 2 or idx == 3 or idx == 4):
        return 1
    elif (idx == 5 or idx == 6 or idx == 7 or idx == 8):
        return 2
    elif (idx == 9 or idx == 10 or idx == 11 or idx == 12):
        return 3
    elif (idx == 13 or idx == 14 or idx == 15 or idx == 16 or idx == 17 or idx == 18 or idx == 19 or idx == 20):
        return 4
    
def acids_to_channels(seq):
    """creates 4*4 feature tensor
       every group corresponds to a special channel. Firstly we encode protein into a seqience of indices
       of amino acids array.
       Eg: 2348
       In every "subdimension - channel" we leave just idxs of elements belonging to this group
       Eg 2 and 3 belongs to the first group, 4 to 2d and 8 to 4-th:
       2300
       0040
       0000
       0008
    """
    amino_seq = seq_to_vec(seq)
    tensor = np.zeros((4, 4))
    for group in range(4):
        for elem in range(4):
            if(group_amino(amino_seq[elem]) == group):
                tensor[group, elem] = amino_seq[elem]
    return tensor
    



In [109]:
X = pd.read_csv("data/train.csv")


def transform_data(X):
    '''transforms data matrix and creates 8 features:
       4 with amino acid name number and 4 with side chain group
    '''
    X['first_acid'] = X['Sequence'].transform(lambda x: acids.index(x[0]))
    X['second_acid'] = X['Sequence'].transform(lambda x: acids.index(x[1]))
    X['third_acid'] = X['Sequence'].transform(lambda x: acids.index(x[2]))
    X['fourth_acid'] = X['Sequence'].transform(lambda x: acids.index(x[3]))

    X['first_group'] = X['Sequence'].transform(lambda x: group_amino(acids.index(x[0])))
    X['second_group'] = X['Sequence'].transform(lambda x: group_amino(acids.index(x[1])))
    X['third_group'] = X['Sequence'].transform(lambda x: group_amino(acids.index(x[2])))
    X['fourth_group'] = X['Sequence'].transform(lambda x: group_amino(acids.index(x[3])))
    return X



In [104]:
X = pd.read_csv("data/train.csv")
X = transform_data(X)
X_data = X.drop(columns = ['Sequence', 'Active'])
y_all = X['Active'].to_numpy()

In [157]:
categoricals  = ['first_acid', 'second_acid', 'third_acid', 'fourth_acid', 'first_group', 'second_group',
                'third_group', 'fourth_group']

x_train = X_data
y_train = y_all
score_summary = []
indexes_of_categories = [x_train.columns.get_loc(col) for col in categoricals]

In [171]:
mdl = LGBMClassifier(categorical_feature=indexes_of_categories)

gridParams = {
    'n_estimators': [40, 100, 200, 500, 1000],
    'reg_alpha' : [0.001, 0.1, 1, 10, 100],
    'reg_lambda' : [0.1, 1, 10, 30],
    'learning_rate': [0.0001, 0.005],
    'num_leaves': [16, 31, 63],
    }

grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=5,
                    n_jobs=2)

#select best features for lightgbm using a grid search
grid.fit(x_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

In [177]:
#Pipeline of our model. We select subspace of features (here all features) and LGBMClassifier classifier
selector = SelectPercentile(percentile=100)
lbm = LGBMClassifier(reg_alpha=0.1, reg_ambda=1, n_estimators=1000, categorical_feature=indexes_of_categories)
pipeline = make_pipeline(selector, lbm)
#0.1, 50 - 93 place

In [178]:
#cross validation with 5 folds and f1 metric score
scores = cross_validate(pipeline, x_train, y_train,
                            scoring='f1',
                            return_train_score=True,
                            cv=5,  # Already include stratified folds
                            return_estimator=True,
                            n_jobs=-1)  # Add parallelism to speed up validation
print("Test scores:\t", scores['test_score'],
        '\nTrain scores:\t', scores['train_score'])
print("Test: %0.4f (+/- %0.4f)" % (scores['test_score'].mean(), scores['test_score'].std() * 2))
score_summary.append(scores['test_score'].mean())
print("Train: %0.4f (+/- %0.4f)" % (scores['train_score'].mean(), scores['train_score'].std() * 2))

pipeline.fit(x_train, y_train)
# y_pred = pipeline.predict_proba(x_test)[:,1]

Test scores:	 [0.90722892 0.72572816 0.89105295 0.75933864 0.73690548] 
Train scores:	 [1.         0.84240862 1.         0.83501887 0.84065851]
Test: 0.8041 (+/- 0.1571)
Train: 0.9036 (+/- 0.1575)


/Users/daniil/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  .format(key))


Pipeline(memory=None,
         steps=[('selectpercentile',
                 SelectPercentile(percentile=90,
                                  score_func=<function f_classif at 0x11c519d40>)),
                ('lgbmclassifier',
                 LGBMClassifier(boosting_type='gbdt',
                                categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7],
                                class_weight=None, colsample_bytree=1.0,
                                importance_type='split', learning_rate=0.1,
                                max_depth=-1, min_child_samples=20,
                                min_child_weight=0.001, min_split_gain=0.0,
                                n_estimators=1000, n_jobs=-1, num_leaves=31,
                                objective=None, random_state=None,
                                reg_alpha=0.1, reg_ambda=1, reg_lambda=0.0,
                                silent=True, subsample=1.0,
                                subsample_for_bin=200000, subsample_

In [175]:
#reading & transforming test dataset and getting a prediction

x_final = pd.read_csv('data/test.csv')
x_final = transform_data(x_final)
x_final = x_final.drop(columns = ['Sequence'])
# x_final['Sequence'] = x_final['Sequence'].transform(lambda x: seq_to_hot(x))
# x_final = np.stack(x_final['Sequence'].to_numpy())
# x_final = np.reshape(x_final, (x_final.shape[0], -1))
answer = pipeline.predict(x_final)
np.savetxt("answer_lgbm_grid_search.csv", answer, delimiter=",", fmt='%d')